# Polars DataFrame library

The Polars syntax is very close to PySpark and much more efficient than Pandas.

In [14]:
from typing import List
import numpy as np
import pandas as pd
import polars as pl

## Aggregating with missing values

In [2]:
df = pl.DataFrame({
    'Record': [1,1,1,2,2],
    'Value': [np.nan, 1,2,3,np.nan]
})

df

Record,Value
i64,f64
1,NaN
1,1
1,2
2,3
2,NaN


Basic aggregation does not ignore nan or null values.

In [8]:
df.groupby('Record').agg([
    pl.col('Value').max().alias('Value_max'),
    pl.col('Value').min().alias('Value_min'),
    pl.col('Value').mean().alias('Value_mean')
]).sort('Record')

Record,Value_max,Value_min,Value_mean
i64,f64,f64,f64
1,2,1,NaN
2,NaN,NaN,NaN


But you can apply filtering during aggregation. Here, we calculate the max/min/mean per group but we don't want to include nan values so we apply a filter on the column inside the group.

Docs:
- [GroupBy - Filtering](https://pola-rs.github.io/polars-book/user-guide/dsl/groupby.html#filtering)
- [Expr.is_not_null](https://pola-rs.github.io/polars/py-polars/html/reference/api/polars.Expr.is_not_null.html)

In [12]:
col = pl.col('Value').filter(pl.col('Value').is_not_nan())

(
    df
    .lazy()
    .groupby('Record')
    .agg([
        col.max().alias('Value_max'),
        col.min().alias('Value_min'),
        col.mean().alias('Value_mean'),
    ])
    .sort('Record')
    .collect()
)

Record,Value_max,Value_min,Value_mean
i64,f64,f64,f64
1,2,1,1.5
2,3,3,3


We can also define a reusable `finite` function that selects non-null values.

In [13]:
def finite(var: str) -> pl.Expr:
    return pl.col(var).filter(pl.col(var).is_not_null() & pl.col(var).is_not_nan())

## Wide to tall table statistics

It's common to calculate statistics (e.g., mean, std) from a wide table. But we want the result as a tall table with columns `[Variable, Mean, Std]`. We can first melth the wide table to a tall table and then group by the variable column to aggregate statistics.

In [75]:
def calculate_stats(df: pl.DataFrame, value_vars: List[str]) -> pl.DataFrame:
    stats = (
        df
        .melt(id_vars=[], value_vars=value_vars)
        .drop_nulls()
        .groupby('variable')
        .agg([
            pl.col('value').mean().alias('Mean'),
            pl.col('value').std().alias('Std')
        ])
        .rename({'variable':'Variable'})
    )
    return stats

In [76]:
df = pl.DataFrame({
    'Record': pl.Series([1.,1.,1.,2.,2.]),
    'Dataset': ['a','b','a','a','b'],
    'HR': [np.nan,90.,80.,85.,np.nan],
    'SBP': [130.,120.,125.,100.,80.]
})

df

Record,Dataset,HR,SBP
f64,str,f64,f64
1,"""a""",NaN,130
1,"""b""",90,120
1,"""a""",80,125
2,"""a""",85,100
2,"""b""",NaN,80


In [81]:
value_vars=['HR','SBP']
calculate_stats(df,value_vars)

Variable,Mean,Std
str,f64,f64
"""SBP""",111,20.73644135332772
"""HR""",NaN,NaN


Use the `melt(id_vars, value_vars)` function to transform a wide table to a tall table by stacking each column. Returns columns `[variable, value]`. The tall table will include missing values so we need to add a filter to remove those rows.

> **Note**: nan (np.nan) and null (None) values are treated differently in polars. Use the filter expressions `is_not_null` and `is_not_nan` to remove missing values.

Docs:
- [pl.DataFrame.melt](https://pola-rs.github.io/polars/py-polars/html/reference/api/polars.DataFrame.melt.html)

In [89]:
value_vars=['HR','SBP']
(
    df
    .melt(id_vars=[], value_vars=value_vars)
    .filter(pl.col('value').is_not_nan() & pl.col('value').is_not_null())
)

variable,value
str,f64
"""HR""",90
"""HR""",80
"""HR""",85
"""SBP""",130
"""SBP""",120
"""SBP""",125
"""SBP""",100
"""SBP""",80
